In [52]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [53]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [54]:
# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [55]:
print(len(training_set), len(test_set))

79999 19999


In [56]:
training_set[:][:5]

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712],
       [        1,         7,         4, 875071561]])

In [57]:
print(users.shape, movies.shape)
# np.zeros(users.shape[0])

(6040, 5) (3883, 3)


In [58]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [59]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [60]:
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [61]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [62]:
ls = torch.FloatTensor([1,2,3,4])
rm = Variable(ls).unsqueeze(0)
torch.sum(rm.data)

tensor(10.)

In [63]:
torch.sum(rm.data > 2) > 3

tensor(False)

In [64]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0.
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(target)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.7723052708701907
epoch: 2 loss: 1.096670960506384
epoch: 3 loss: 1.0534786480785725
epoch: 4 loss: 1.0386633879928335
epoch: 5 loss: 1.0308984219715702
epoch: 6 loss: 1.0265914606053006
epoch: 7 loss: 1.0237834773368586
epoch: 8 loss: 1.0218539890237166
epoch: 9 loss: 1.020680079839534
epoch: 10 loss: 1.0196458196686469
epoch: 11 loss: 1.018859316874469
epoch: 12 loss: 1.018507061861535
epoch: 13 loss: 1.0179205016115425
epoch: 14 loss: 1.0175630164412297
epoch: 15 loss: 1.0170758074728121
epoch: 16 loss: 1.0170595704444976
epoch: 17 loss: 1.0166437503553314
epoch: 18 loss: 1.0164027170206418
epoch: 19 loss: 1.0162592232804881
epoch: 20 loss: 1.0161293242959264
epoch: 21 loss: 1.0160460080408988
epoch: 22 loss: 1.0158371610105679
epoch: 23 loss: 1.0159481103492853
epoch: 24 loss: 1.0157432146364664
epoch: 25 loss: 1.0155279604085676
epoch: 26 loss: 1.0154158165634366
epoch: 27 loss: 1.0155154722755455
epoch: 28 loss: 1.0148353210715333
epoch: 29 loss: 1.013296227338172

In [66]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

/Users/rkumar/anaconda2/envs/py3/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: 0.9534503771054728
